# Part 2.1: Python OOP for Deep Learning — The Formula 1 Edition

Object-Oriented Programming is essential for deep learning because:
- PyTorch models are classes (`nn.Module`)
- Datasets are classes (`torch.utils.data.Dataset`)
- Training loops use objects with state
- Clean, reusable code requires good OOP design

**F1 analogy:** An F1 team is a masterclass in object-oriented thinking. A `Car` is a class — it defines what every car *has* (engine, tires, aero package) and what every car *does* (accelerate, brake, turn). But each physical car on the grid is an *instance* — Verstappen's car and his teammate's car follow the same blueprint but carry different settings, different wear, different telemetry data. The team itself is a composition of objects: drivers, engineers, pit crew, strategy software. Inheritance is everywhere: a `SafetyCar` is a kind of `Vehicle`, but with different rules. And the FIA's technical regulations? Those are like type hints and decorators — constraints that validate and wrap behavior without changing the core engineering.

## Learning Objectives
- [ ] Design clean class hierarchies
- [ ] Use magic methods to create Pythonic APIs
- [ ] Write and use decorators
- [ ] Add type hints for better code documentation

---

In [ ]:
# Setup - run this cell first
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('seaborn-v0_8-whitegrid')
np.random.seed(42)

### What this means:

**Classes** are like blueprints for building things. Think of a class as an F1 car design specification — it defines the aerodynamics, engine layout, and suspension geometry, but each physical car built from that spec is a separate object with its own wear, setup, and telemetry history.

**F1 analogy:** The `RB20` class is the blueprint. When Red Bull builds chassis #1 for Verstappen and chassis #2 for Perez, those are two *instances*. They share the same design (class), but each accumulates its own mileage, crash damage, and component life. Changing Verstappen's front wing angle doesn't change Perez's.

In deep learning:
- The `nn.Module` class is the car design spec for all neural networks
- When you write `model = MyNetwork()`, you're building one specific car from that spec
- Each model you create has its own weights, even though they follow the same blueprint

## 1. Classes and Objects

A **class** is a blueprint for creating objects. An **object** is an instance of a class.

### Why Classes Matter in Deep Learning

| PyTorch Concept | Implemented As | F1 Parallel |
|-----------------|----------------|-------------|
| Neural network | Class extending `nn.Module` | Car design spec (aero, engine, suspension) |
| Dataset | Class extending `Dataset` | Season's race calendar with track data |
| Optimizer | Class with `step()` and `zero_grad()` | Race strategist adjusting setup each lap |
| Loss function | Class with `forward()` method | Gap to leader — the number you want to minimize |
| Data loader | Class with `__iter__()` method | Pit crew feeding tires and fuel in batches |

### The F1 Connection

Every F1 car on the grid is an *instance* of a design class. The constructor (`__init__`) is the factory build — it sets the initial engine mode, tire compound, fuel load, and driver assignment. Methods like `accelerate()`, `brake()`, and `change_tires()` define what the car can *do*. This is exactly how PyTorch models work: `__init__` builds the layers, and `forward()` defines the computation.

In [ ]:
# Basic class structure — F1 telemetry point
class TelemetryPoint:
    """A single telemetry reading from an F1 car's sensors."""
    
    def __init__(self, n_sensors):
        """Initialize telemetry with random sensor calibrations.
        
        Args:
            n_sensors: Number of sensor channels (speed, throttle, brake, g-force, etc.)
        """
        import numpy as np
        self.weights = np.random.randn(n_sensors)  # sensor calibration weights
        self.bias = 0.0  # baseline offset
        
    def forward(self, x):
        """Compute weighted telemetry reading (same math as a neuron!)."""
        import numpy as np
        z = np.dot(self.weights, x) + self.bias
        return 1 / (1 + np.exp(-z))  # Sigmoid: normalizes to 0-1 range

# Create an instance (object)
telemetry = TelemetryPoint(n_sensors=3)
print(f"Sensor weights: {telemetry.weights}")
print(f"Baseline offset: {telemetry.bias}")

# Read a telemetry snapshot: [speed_ratio, throttle_position, brake_pressure]
import numpy as np
sensor_reading = np.array([1.0, 2.0, 3.0])
output = telemetry.forward(sensor_reading)
print(f"Output for sensor reading {sensor_reading}: {output:.4f}")

### Deep Dive: Understanding `self`

`self` refers to the specific instance of the class. It's how the object accesses its own data.

**F1 analogy:** When a race engineer says "check your tire temps," the word "your" is `self` — it refers to *this specific car*. Verstappen's engineer is talking to Verstappen's car, not Perez's.

```python
car1 = TelemetryPoint(3)  # self = car1 inside methods
car2 = TelemetryPoint(3)  # self = car2 inside methods
```

Each object has its own `weights` and `bias` — they don't share! Just like each car has its own telemetry data.

| Term | Meaning | F1 Parallel |
|------|--------|-------------|
| `self.weights` | Instance attribute (each object has its own) | This car's sensor calibration |
| `self.forward(x)` | Instance method (operates on this object's data) | This car's telemetry computation |
| `TelemetryPoint.forward` | The method definition in the class | The spec for how any car computes telemetry |

In [ ]:
# Demonstrating that each instance has its own state
np.random.seed(42)
car1 = TelemetryPoint(3)  # Verstappen's car
car2 = TelemetryPoint(3)  # Perez's car

print("car1 (Verstappen) sensor weights:", car1.weights)
print("car2 (Perez) sensor weights:", car2.weights)
print("\nThey're different! Each car has its own sensor calibration.")

# Modify one, the other is unaffected — just like adjusting one car's setup
car1.bias = 1.0
print(f"\ncar1.bias = {car1.bias}  (adjusted Verstappen's baseline)")
print(f"car2.bias = {car2.bias}  (Perez's unchanged)")

In [ ]:
# Visualization: Object Memory Layout — F1 Cars in the Garage
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Left: Two separate car objects
ax = axes[0]
ax.set_xlim(0, 10)
ax.set_ylim(0, 10)
ax.axis('off')
ax.set_title('Two F1 Car Objects in Memory', fontsize=12, fontweight='bold')

# Object 1 box
rect1 = plt.Rectangle((0.5, 5), 4, 4.5, fill=True, facecolor='lightblue', 
                        edgecolor='black', linewidth=2)
ax.add_patch(rect1)
ax.text(2.5, 9, 'car1 (VER)', ha='center', va='center', fontsize=11, fontweight='bold')
ax.text(2.5, 8.2, 'weights: [0.49, -0.13, 0.64]', ha='center', fontsize=9)
ax.text(2.5, 7.4, 'bias: 1.0', ha='center', fontsize=9)
ax.text(2.5, 6.5, 'forward: <method>', ha='center', fontsize=9, style='italic')
ax.text(2.5, 5.4, 'id: 0x7f...a1b0', ha='center', fontsize=8, color='gray')

# Object 2 box
rect2 = plt.Rectangle((5.5, 5), 4, 4.5, fill=True, facecolor='lightgreen', 
                        edgecolor='black', linewidth=2)
ax.add_patch(rect2)
ax.text(7.5, 9, 'car2 (PER)', ha='center', va='center', fontsize=11, fontweight='bold')
ax.text(7.5, 8.2, 'weights: [1.52, -0.23, 0.54]', ha='center', fontsize=9)
ax.text(7.5, 7.4, 'bias: 0.0', ha='center', fontsize=9)
ax.text(7.5, 6.5, 'forward: <method>', ha='center', fontsize=9, style='italic')
ax.text(7.5, 5.4, 'id: 0x7f...c2d0', ha='center', fontsize=8, color='gray')

ax.text(5, 3.5, 'Each car has its own copy\nof instance attributes!', 
        ha='center', fontsize=10, style='italic')

# Right: Class vs Instance attributes — Team regulations vs car setup
ax = axes[1]
ax.set_xlim(0, 10)
ax.set_ylim(0, 10)
ax.axis('off')
ax.set_title('Team Spec (Class) vs Car Setup (Instance)', fontsize=12, fontweight='bold')

# Class (shared) — like FIA regulations or team design spec
rect_class = plt.Rectangle((2, 6.5), 6, 2.5, fill=True, facecolor='lightyellow', 
                             edgecolor='black', linewidth=2)
ax.add_patch(rect_class)
ax.text(5, 8.5, 'Team (class)', ha='center', fontweight='bold', fontsize=11)
ax.text(5, 7.5, 'car_count = 3  (shared by all)', ha='center', fontsize=10)

# Instances — individual cars
for i, (x, color, name) in enumerate([(1.5, 'lightblue', 'car_FP1'), 
                                        (5, 'lightgreen', 'car_FP2'),
                                        (8.5, 'lightcoral', 'car_Race')]):
    rect = plt.Rectangle((x-1.2, 2), 2.4, 3, fill=True, facecolor=color, 
                          edgecolor='black', linewidth=1.5)
    ax.add_patch(rect)
    ax.text(x, 4.5, name, ha='center', fontweight='bold', fontsize=10)
    ax.text(x, 3.8, f'id = {i}', ha='center', fontsize=9)
    ax.text(x, 3.1, f'downforce = {[350, 450, 400][i]}kg', ha='center', fontsize=8)
    # Arrow to class
    ax.annotate('', xy=(x, 6.5), xytext=(x, 5),
                arrowprops=dict(arrowstyle='->', color='gray', lw=1.5))

ax.text(5, 1, 'Instance attributes (setup) are unique;\nclass attributes (team spec) are shared', 
        ha='center', fontsize=10, style='italic')

plt.tight_layout()
plt.show()

### Class Attributes vs Instance Attributes

| Type | Defined | Shared? | Use Case | F1 Parallel |
|------|---------|---------|----------|-------------|
| Class attribute | In class body | Yes, by all instances | Constants, counters | FIA regulations (same for all cars) |
| Instance attribute | In `__init__` with `self.` | No, each instance has own | Object-specific data | Car setup (unique per chassis) |

### What this means:

**Inheritance** is like the FIA vehicle classification system. Every car on the grid is a `Vehicle`, but an `F1Car` inherits from `Vehicle` and adds open-wheel aero, DRS, and hybrid power units. A `SafetyCar` also inherits from `Vehicle` but has flashing lights and speed limiters instead. Both get basic vehicle behavior (steering, braking) for free — they only define what's unique.

**F1 analogy:** Think of tire compounds. There's a base `Tire` class that defines grip, degradation, and temperature behavior. `SoftTire`, `MediumTire`, and `HardTire` all inherit from `Tire` but override the specific grip and durability characteristics. You don't rewrite the entire tire physics for each compound — you inherit the common behavior and customize the differences.

In deep learning:
- `nn.Module` is the "parent" that knows how to track parameters, move to GPU, save/load, etc.
- Your custom model is the "child" that inherits all those abilities
- You only need to write what's unique (your architecture), not reinvent parameter tracking!

In [ ]:
# Visualization: Class Hierarchy Diagram — F1 Vehicle Inheritance
fig, ax = plt.subplots(figsize=(10, 6))
ax.set_xlim(0, 10)
ax.set_ylim(0, 8)
ax.axis('off')
ax.set_title('Class Hierarchy: Inheritance in F1 and Deep Learning', fontsize=14, fontweight='bold')

# Draw boxes
def draw_box(ax, x, y, text, color='lightblue'):
    box = plt.Rectangle((x-0.8, y-0.3), 1.6, 0.6, fill=True, 
                         facecolor=color, edgecolor='black', linewidth=2)
    ax.add_patch(box)
    ax.text(x, y, text, ha='center', va='center', fontsize=10, fontweight='bold')

# Parent class
draw_box(ax, 5, 7, 'Vehicle', 'lightyellow')

# Child classes (level 1) — different vehicle types
draw_box(ax, 2, 5, 'F1Car', 'lightblue')
draw_box(ax, 5, 5, 'SafetyCar', 'lightblue')
draw_box(ax, 8, 5, 'MedicalCar', 'lightblue')

# Specific cars (level 2) — instances/further specialization
draw_box(ax, 2, 3, 'RedBullRB20', 'lightgreen')
draw_box(ax, 5, 3, 'Mercedes AMG', 'lightgreen')
draw_box(ax, 8, 3, 'Aston Martin', 'lightgreen')

# Draw inheritance arrows
for x in [2, 5, 8]:
    ax.annotate('', xy=(x, 6.7), xytext=(x, 5.3),
                arrowprops=dict(arrowstyle='->', color='black', lw=2))
    ax.annotate('', xy=(x, 4.7), xytext=(x, 3.3),
                arrowprops=dict(arrowstyle='->', color='black', lw=2))

# Legend
ax.text(5, 1.5, 'Arrows show "inherits from" relationship', ha='center', fontsize=10, style='italic')
ax.text(5, 1, 'Yellow = Base Vehicle | Blue = Vehicle types | Green = Specific implementations', 
        ha='center', fontsize=9)

plt.tight_layout()
plt.show()

In [ ]:
# Class attributes vs instance attributes — F1 Team example
class Team:
    """An F1 constructor team."""
    
    # Class attribute - shared by all teams (like FIA rules)
    total_teams = 0
    
    def __init__(self, n_staff):
        # Instance attributes - unique to each team
        self.n_staff = n_staff
        self.team_id = Team.total_teams
        Team.total_teams += 1  # Increment the shared counter

# Create teams
red_bull = Team(800)
mercedes = Team(900)
ferrari = Team(850)

print(f"Red Bull: id={red_bull.team_id}, staff={red_bull.n_staff}")
print(f"Mercedes: id={mercedes.team_id}, staff={mercedes.n_staff}")
print(f"Ferrari:  id={ferrari.team_id}, staff={ferrari.n_staff}")
print(f"\nTotal teams on the grid: {Team.total_teams}")

---

## 2. Inheritance

**Inheritance** lets you create a new class based on an existing class. The new class "inherits" all the methods and attributes of the parent.

### Why Inheritance Matters in Deep Learning

```python
class MyModel(nn.Module):      # Inherit from nn.Module
class MyDataset(Dataset):       # Inherit from Dataset
class MyOptimizer(Optimizer):   # Inherit from Optimizer
```

You get all the PyTorch machinery for free, then customize what you need!

### The F1 Connection

Inheritance in F1 works exactly the same way. Every tire compound inherits from a base `Tire` class — they all have grip, degradation curves, and temperature windows. But a `SoftTire` overrides the grip coefficient to be higher and the durability to be lower. You don't rewrite the tire physics from scratch for each compound. In PyTorch, when you write `class MyModel(nn.Module)`, you inherit GPU support, parameter tracking, serialization, and gradient computation — then you only define your specific `forward()` method.

### What this means:

**Magic methods** are special methods that Python calls automatically when you use operators or built-in functions. The double underscores ("dunders") tell Python "this is special."

**F1 analogy:** Magic methods are like the standardized interfaces on an F1 car. The steering wheel *must* have specific paddle positions, the fuel connector *must* fit the refueling rig, the data port *must* speak the FIA telemetry protocol. You don't call these interfaces manually — they activate automatically when the right situation occurs (pit stop, data download, safety car deployment). Similarly:
- When you write `len(race_results)`, Python secretly calls `race_results.__len__()`
- When you write `car(telemetry)`, Python secretly calls `car.__call__(telemetry)`
- You're teaching Python how to treat YOUR objects like built-in types!

In [ ]:
# Base class — like a base Tire compound
class Tire:
    """Base class for F1 tire compounds (also demonstrates activation function pattern)."""
    
    def __init__(self, name):
        self.name = name
        
    def grip_curve(self, x):
        """Compute grip response — must be implemented by subclass."""
        raise NotImplementedError("Subclasses must implement grip_curve()")
    
    def __repr__(self):
        return f"{self.__class__.__name__}()"


# Child classes — specific tire compounds inherit from Tire
class SoftTire(Tire):
    """Soft compound: high grip, fast degradation (like ReLU — aggressive response)."""
    
    def __init__(self):
        super().__init__("Soft (C5)")  # Call parent's __init__
        
    def grip_curve(self, x):
        return np.maximum(0, x)  # ReLU: aggressive, linear response


class MediumTire(Tire):
    """Medium compound: balanced grip (like Sigmoid — smooth, bounded response)."""
    
    def __init__(self):
        super().__init__("Medium (C3)")
        
    def grip_curve(self, x):
        return 1 / (1 + np.exp(-x))  # Sigmoid: smooth transition


class HardTire(Tire):
    """Hard compound: conservative, durable (like Tanh — symmetric, conservative)."""
    
    def __init__(self):
        super().__init__("Hard (C1)")
        
    def grip_curve(self, x):
        return np.tanh(x)  # Tanh: conservative, symmetric


# Use them — these are also activation functions!
tire_compounds = [SoftTire(), MediumTire(), HardTire()]
x = np.linspace(-3, 3, 100)

import matplotlib.pyplot as plt
plt.figure(figsize=(10, 4))
colors = ['red', 'gold', 'white']
for tire, color in zip(tire_compounds, colors):
    edgecolor = 'black' if color == 'white' else color
    plt.plot(x, tire.grip_curve(x), label=tire.name, linewidth=2, color=edgecolor)
plt.xlabel('Input (throttle demand / neural activation)')
plt.ylabel('Response (grip level / activation output)')
plt.title('Tire Compound Response Curves = Activation Functions (via inheritance)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

# All share the same interface!
print("All tire compounds share the same interface:")
for tire in tire_compounds:
    print(f"  {tire} -> grip_curve(0) = {tire.grip_curve(np.array([0]))[0]:.4f}")

### Deep Dive: `super()` and Method Resolution Order

`super()` calls the parent class's method. This is essential when you want to extend (not replace) the parent's behavior.

**F1 analogy:** When a new tire compound is developed, it doesn't start from scratch. The engineers call `super().__init__()` — they initialize the base rubber chemistry, the internal structure, and the temperature model from the parent `Tire` class. *Then* they add compound-specific modifications like softer rubber polymers or harder carcass construction.

```python
class SoftTire(Tire):
    def __init__(self):
        super().__init__("Soft")  # Initialize base tire first!
        self.grip_multiplier = 1.3  # Then add soft-specific stuff
```

**In PyTorch, you MUST call `super().__init__()`** in your model's `__init__`!

In [ ]:
# Simulating PyTorch's nn.Module pattern — like an F1 car's ECU (Electronic Control Unit)
class Module:
    """Simplified version of nn.Module — think of it as the car's ECU base system."""
    
    def __init__(self):
        self._parameters = {}
        self._modules = {}
        
    def register_parameter(self, name, value):
        self._parameters[name] = value
        
    def parameters(self):
        """Return all parameters — like listing all tunable settings in the ECU."""
        params = list(self._parameters.values())
        for module in self._modules.values():
            params.extend(module.parameters())
        return params
    
    def __call__(self, x):
        return self.forward(x)


class Linear(Module):
    """Linear layer: y = xW + b — a single processing stage in the car's data pipeline."""
    
    def __init__(self, in_features, out_features):
        super().__init__()  # MUST call parent's __init__!
        
        # Initialize weights — like calibrating sensor mappings
        self.weight = np.random.randn(in_features, out_features) * 0.01
        self.bias = np.zeros(out_features)
        
        # Register as parameters
        self.register_parameter('weight', self.weight)
        self.register_parameter('bias', self.bias)
        
    def forward(self, x):
        return x @ self.weight + self.bias


# Use it — process a batch of telemetry readings
layer = Linear(10, 5)
x = np.random.randn(3, 10)  # 3 laps, 10 sensor channels each
output = layer(x)  # Calls __call__ -> forward

print(f"Input shape (laps x sensors): {x.shape}")
print(f"Output shape (laps x features): {output.shape}")
print(f"Number of parameter sets: {len(layer.parameters())}")
print(f"Weight shape: {layer.weight.shape}")
print(f"Bias shape: {layer.bias.shape}")

---

## 3. Magic Methods (Dunder Methods)

**Magic methods** (also called "dunder" methods for "double underscore") let you define how your objects behave with Python's built-in operations.

### Why Magic Methods Matter in Deep Learning

| Magic Method | Enables | PyTorch Example | F1 Parallel |
|--------------|---------|----------------|-------------|
| `__init__` | Creating objects | `model = MyModel()` | Building a car in the factory |
| `__call__` | Calling like function | `output = model(input)` | Sending the car out on track |
| `__len__` | `len()` function | `len(dataset)` | Number of laps in a stint |
| `__getitem__` | Indexing with `[]` | `dataset[0]` | Looking up lap 17's telemetry |
| `__iter__` | For loops | `for batch in dataloader:` | Iterating through race laps |
| `__repr__` | Nice printing | `print(model)` | Pit wall display showing car status |
| `__add__` | The `+` operator | `tensor1 + tensor2` | Combining stint times: S1 + S2 = total |
| `__lt__` | The `<` operator | `result1 < result2` | Championship standings comparison |

In [ ]:
# A RaceResult class demonstrating magic methods — like a lap time or stint record
class RaceResult:
    """A race result / stint time demonstrating magic methods (same math as a Tensor)."""
    
    def __init__(self, data):
        """Called when you do: r = RaceResult(data)"""
        self.data = np.array(data)
        
    def __repr__(self):
        """Called when you do: print(r) — like the pit wall display"""
        return f"RaceResult({self.data.tolist()})"
    
    def __len__(self):
        """Called when you do: len(r) — how many laps/sectors in this result"""
        return len(self.data)
    
    def __getitem__(self, idx):
        """Called when you do: r[idx] — look up a specific lap"""
        return RaceResult(self.data[idx])
    
    def __add__(self, other):
        """Called when you do: r1 + r2 — combine stint times"""
        if isinstance(other, RaceResult):
            return RaceResult(self.data + other.data)
        return RaceResult(self.data + other)
    
    def __mul__(self, other):
        """Called when you do: r1 * r2 — scale times (e.g., fuel correction)"""
        if isinstance(other, RaceResult):
            return RaceResult(self.data * other.data)
        return RaceResult(self.data * other)
    
    def __matmul__(self, other):
        """Called when you do: r1 @ r2 — dot product (setup-track alignment)"""
        return RaceResult(self.data @ other.data)
    
    def __lt__(self, other):
        """Called when you do: r1 < r2 — championship standings comparison"""
        if isinstance(other, RaceResult):
            return np.sum(self.data) < np.sum(other.data)
        return np.sum(self.data) < other
    
    @property
    def shape(self):
        """Access like r.shape (not r.shape()) — just like numpy"""
        return self.data.shape


# Demonstrate magic methods with race data
stint1 = RaceResult([91.2, 90.8, 91.5])  # Lap times for stint 1
stint2 = RaceResult([92.1, 91.5, 92.0])  # Lap times for stint 2

print("__repr__ (pit wall display):", stint1)
print("__len__ (laps in stint):", len(stint1))
print("__getitem__ (lap 1 time):", stint1[0])
print("__add__ (combine stints):", stint1 + stint2)
print("__mul__ (fuel-corrected):", stint1 * 0.98)
print("__lt__ (faster stint?):", stint1 < stint2, "(lower total = faster)")
print("scalar add (penalty +5s):", stint1 + 5)
print("shape property:", stint1.shape)

### `__call__`: Making Objects Callable

This is **THE** most important magic method for deep learning. It lets you call an object like a function:

```python
model = MyModel()    # __init__
output = model(x)    # __call__ -> forward
```

**F1 analogy:** `__call__` is the green light at the end of pit lane. When you "call" the car (send it onto the track), the car doesn't just drive — it runs a whole sequence: warm up tires, engage DRS detection, start telemetry logging, *then* execute the actual lap (`forward()`), then collect post-lap data. PyTorch's `nn.Module.__call__` works the same way:
1. Calls hooks (if any) — pre-lap checks
2. Calls your `forward()` method — the actual lap
3. Calls more hooks — post-lap data collection
4. Returns the result

In [ ]:
# An F1 car's telemetry processing pipeline — demonstrating __call__
class TelemetryPipeline:
    """Neural network that processes F1 telemetry data, demonstrating __call__."""
    
    def __init__(self, layer_sizes):
        self.layers = []
        for i in range(len(layer_sizes) - 1):
            self.layers.append(Linear(layer_sizes[i], layer_sizes[i+1]))
        self.activation = SoftTire()  # Using our ReLU-equivalent tire compound!
            
    def forward(self, x):
        """Forward pass — process telemetry through all layers."""
        for i, layer in enumerate(self.layers):
            x = layer(x)
            # Apply activation to all but last layer
            if i < len(self.layers) - 1:
                x = self.activation.grip_curve(x)
        return x
    
    def __call__(self, x):
        """Makes the pipeline callable — like sending the car out on track."""
        return self.forward(x)
    
    def __repr__(self):
        """Pit wall display — shows pipeline architecture."""
        layers_str = "\n  ".join([f"Linear({l.weight.shape[0]} -> {l.weight.shape[1]})" 
                                   for l in self.layers])
        return f"TelemetryPipeline(\n  {layers_str}\n)"


# Create and use the pipeline
pipeline = TelemetryPipeline([784, 128, 64, 10])
print(pipeline)

# Forward pass — note we call pipeline(x), not pipeline.forward(x)
x = np.random.randn(32, 784)  # Batch of 32 telemetry snapshots (784 sensor channels)
output = pipeline(x)  # This calls __call__ -> forward

print(f"\nInput shape (snapshots x sensors): {x.shape}")
print(f"Output shape (snapshots x predictions): {output.shape}")

### `__getitem__` and `__len__`: Building a Dataset

These methods let you create custom datasets that work with PyTorch's DataLoader.

**F1 analogy:** Think of a season's worth of race data. `__len__` tells you how many races are in the calendar (e.g., 24). `__getitem__` lets you pull up the data for any specific race by index — `season[5]` gets you the Monaco Grand Prix telemetry. This is exactly how PyTorch datasets work.

In [ ]:
class Dataset:
    """Base dataset class (like torch.utils.data.Dataset)."""
    
    def __len__(self):
        raise NotImplementedError
        
    def __getitem__(self, idx):
        raise NotImplementedError


class SeasonTelemetryDataset(Dataset):
    """A season's worth of F1 telemetry data for demonstration."""
    
    def __init__(self, n_laps, n_sensors, n_tire_compounds):
        # Each lap has sensor readings (features) and tire compound used (label)
        self.X = np.random.randn(n_laps, n_sensors)  # Telemetry readings
        self.y = np.random.randint(0, n_tire_compounds, n_laps)  # Tire compound: 0=soft, 1=medium, 2=hard
        
    def __len__(self):
        """Return number of laps in dataset."""
        return len(self.X)
    
    def __getitem__(self, idx):
        """Return (telemetry, tire_compound) for given lap index."""
        return self.X[idx], self.y[idx]


# Create dataset — a season of telemetry
season_data = SeasonTelemetryDataset(n_laps=1000, n_sensors=10, n_tire_compounds=3)

# Now we can use len() and indexing!
print(f"Total laps in dataset: {len(season_data)}")
print(f"Lap 0: telemetry={season_data[0][0][:3]}..., tire_compound={season_data[0][1]}")
print(f"Last lap: telemetry={season_data[-1][0][:3]}..., tire_compound={season_data[-1][1]}")

# We can iterate through laps!
compound_names = {0: 'Soft', 1: 'Medium', 2: 'Hard'}
print("\nFirst 3 laps:")
for i in range(3):
    telemetry, compound = season_data[i]
    print(f"  Lap {i}: tire compound = {compound_names[compound]}")

### Complete Magic Methods Reference

| Method | Triggered By | Example | F1 Parallel |
|--------|--------------|--------|-------------|
| `__init__(self, ...)` | `obj = Class(...)` | Constructor | Building the car at the factory |
| `__repr__(self)` | `print(obj)`, `repr(obj)` | String representation | Pit wall display |
| `__str__(self)` | `str(obj)` | Human-readable string | TV broadcast graphic |
| `__len__(self)` | `len(obj)` | Length | Number of laps in a stint |
| `__getitem__(self, key)` | `obj[key]` | Indexing | Pull up lap 17's data |
| `__setitem__(self, key, val)` | `obj[key] = val` | Index assignment | Override a lap's data |
| `__call__(self, ...)` | `obj(...)` | Call like function | Send the car onto track |
| `__iter__(self)` | `for x in obj:` | Iteration | Loop through each race |
| `__next__(self)` | `next(obj)` | Next item | Next lap on the calendar |
| `__add__(self, other)` | `obj + other` | Addition | Combine stint times |
| `__sub__(self, other)` | `obj - other` | Subtraction | Gap to leader |
| `__mul__(self, other)` | `obj * other` | Multiplication | Fuel correction factor |
| `__matmul__(self, other)` | `obj @ other` | Matrix multiplication | Setup-track alignment score |
| `__eq__(self, other)` | `obj == other` | Equality | Same finishing position? |
| `__lt__(self, other)` | `obj < other` | Less than | Championship standings |
| `__enter__(self)` | `with obj:` | Context manager enter | Green flag — session starts |
| `__exit__(self, ...)` | End of `with` block | Context manager exit | Checkered flag — session ends |

### What this means:

**Decorators** are like FIA technical regulations wrapped around your car's design — they add validation, timing, and monitoring without changing the core engineering.

**F1 analogy:** Every F1 lap is automatically timed by the official timing system. The driver doesn't need to start and stop a stopwatch — the timing loops embedded in the track surface *decorate* every lap with precise sector times. Similarly, the FIA fuel flow sensor wraps every engine with a validator that checks compliance without the engine needing to know about it. That's what decorators do:
- `@timed` wraps your function to measure how long it takes (timing loops)
- `@validated` wraps your function to check inputs meet regulations (scrutineering)
- `@torch.no_grad()` wraps your function to disable gradient tracking (like switching off data logging to go faster during qualifying sims)

---

## 4. Decorators

A **decorator** modifies or enhances a function without changing its code. It's a function that takes a function and returns a new function.

### Why Decorators Matter in Deep Learning

| Decorator | Use | F1 Parallel |
|-----------|-----|-------------|
| `@property` | Access method like attribute (`model.device`) | `car.tire_wear` computed from laps driven |
| `@staticmethod` | Method that doesn't need `self` | Utility: convert km/h to mph |
| `@classmethod` | Method that operates on class, not instance | `Car.from_telemetry_file(path)` factory method |
| `@torch.no_grad()` | Disable gradient computation (inference) | Race mode: no data logging overhead |
| `@torch.jit.script` | JIT compile for speed | Turbo mode: pre-compiled strategy model |
| Custom `@timed` decorator | Profile your code | Lap timing system (automatic) |
| Custom `@validated` decorator | Check inputs | FIA scrutineering (technical regulation checks) |

In [ ]:
# Visualization: Decorator Flow Diagram — Lap Timing System
fig, ax = plt.subplots(figsize=(12, 6))
ax.set_xlim(0, 12)
ax.set_ylim(0, 8)
ax.axis('off')
ax.set_title('How Decorators Work: @timed (Like F1 Timing Loops)', fontsize=14, fontweight='bold')

# Helper function to draw boxes
def draw_flow_box(ax, x, y, w, h, text, color='lightblue', fontsize=10):
    rect = plt.Rectangle((x, y), w, h, fill=True, facecolor=color, 
                          edgecolor='black', linewidth=2, alpha=0.8)
    ax.add_patch(rect)
    ax.text(x + w/2, y + h/2, text, ha='center', va='center', fontsize=fontsize)

# Step 1: Original function (the raw lap)
draw_flow_box(ax, 0.5, 5.5, 2.5, 1.5, 'drive_lap()', 'lightyellow', 11)
ax.text(1.75, 7.3, '1. Original\nfunction', ha='center', fontsize=9)

# Arrow
ax.annotate('', xy=(3.3, 6.25), xytext=(3, 6.25),
            arrowprops=dict(arrowstyle='->', color='black', lw=2))

# Step 2: @timed decorator (timing loops on track)
draw_flow_box(ax, 3.5, 5, 3, 2.5, '@timed\ndef drive_lap():\n    ...', 'lightcoral', 10)
ax.text(5, 7.8, '2. Apply\ndecorator', ha='center', fontsize=9)

# Arrow
ax.annotate('', xy=(6.8, 6.25), xytext=(6.5, 6.25),
            arrowprops=dict(arrowstyle='->', color='black', lw=2))

# Step 3: Wrapper function (the full timing system)
draw_flow_box(ax, 7, 4.5, 4.5, 3.5, 'timed_lap():\n  cross timing line\n  result = drive_lap()\n  record sector time\n  return result', 
              'lightgreen', 9)
ax.text(9.25, 8.3, '3. Returns wrapped\nfunction', ha='center', fontsize=9)

# What happens when you call
ax.text(6, 2.5, 'When you call drive_lap():', fontsize=11, fontweight='bold')
ax.text(6, 1.8, 'Python actually calls timed_lap() which:', fontsize=10)
ax.text(6, 1.2, '1. Starts sector timer  2. Drives the actual lap  3. Records lap time', fontsize=9)

# Code example box
code_box = plt.Rectangle((0.5, 0.3, ), 5, 1.5, fill=True, facecolor='#f0f0f0', 
                          edgecolor='gray', linewidth=1)
ax.add_patch(code_box)
ax.text(3, 1.35, '@timed', fontsize=10, fontfamily='monospace')
ax.text(3, 0.85, 'def qualify(): ...', fontsize=10, fontfamily='monospace')
ax.text(3, 0.5, '# Same as: qualify = timed(qualify)', fontsize=9, fontfamily='monospace', color='gray')

plt.tight_layout()
plt.show()

In [ ]:
import time
from functools import wraps

# @timed decorator — like the F1 timing loops embedded in the track
def timed(func):
    """Decorator that times function execution — like sector timing loops."""
    @wraps(func)  # Preserves function name and docstring
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f"{func.__name__} lap time: {end - start:.4f} seconds")
        return result
    return wrapper


@timed
def slow_pit_stop():
    """A slow pit stop for demonstration."""
    time.sleep(0.1)
    return "tires changed"


@timed
def compute_telemetry_matrix(n_laps):
    """Process a telemetry matrix — like crunching race data on the pit wall."""
    A = np.random.randn(n_laps, n_laps)
    B = np.random.randn(n_laps, n_laps)
    return A @ B


# Use the decorated functions — timing is automatic!
result1 = slow_pit_stop()
result2 = compute_telemetry_matrix(1000)

In [ ]:
# More useful decorators — F1 engineering tools

def debug(func):
    """Print function arguments and return value — like telemetry logging."""
    @wraps(func)
    def wrapper(*args, **kwargs):
        args_str = ", ".join([repr(a) for a in args])
        kwargs_str = ", ".join([f"{k}={v!r}" for k, v in kwargs.items()])
        all_args = ", ".join(filter(None, [args_str, kwargs_str]))
        print(f"Calling {func.__name__}({all_args})")
        result = func(*args, **kwargs)
        print(f"  -> {result!r}")
        return result
    return wrapper


def validated(max_attempts=3):
    """Retry a function if it raises an exception — like re-scrutineering after a failure."""
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            for attempt in range(max_attempts):
                try:
                    return func(*args, **kwargs)
                except Exception as e:
                    print(f"Scrutineering attempt {attempt + 1} failed: {e}")
                    if attempt == max_attempts - 1:
                        raise
        return wrapper
    return decorator


@debug
def calculate_gap(leader_time, chaser_time):
    """Calculate gap between two drivers."""
    return chaser_time - leader_time

gap = calculate_gap(91.2, 91.8)
gap = calculate_gap(91.2, chaser_time=92.1)

### Built-in Decorators: `@property`, `@staticmethod`, `@classmethod`

**F1 analogy:** `@property` is perfect for values that are *computed* from the car's state — like `tire_wear` (depends on laps driven) or `fuel_load` (decreases each lap). You access them like attributes (`car.tire_wear`), but they're recalculated each time. `@staticmethod` is a utility that doesn't need a specific car instance — like a unit converter. `@classmethod` operates on the team/class level — like counting total cars built.

In [ ]:
class F1Car:
    """An F1 car demonstrating @property, @staticmethod, @classmethod."""
    
    _car_count = 0
    MAX_FUEL_KG = 110  # FIA regulation: max 110kg fuel
    
    def __init__(self, driver_name, initial_fuel_kg=110):
        self.driver_name = driver_name
        self._fuel_kg = initial_fuel_kg
        self._laps_driven = 0
        self._tire_compound = "Medium"
        F1Car._car_count += 1
        
    @property
    def tire_wear(self):
        """Computed from laps driven — access like an attribute, not a method call."""
        # Tire wear increases ~1.5% per lap (simplified model)
        wear_pct = min(100, self._laps_driven * 1.5)
        return wear_pct
    
    @property
    def fuel_load(self):
        """Fuel decreases ~1.7kg per lap — computed on the fly."""
        return max(0, self._fuel_kg - self._laps_driven * 1.7)
    
    @property
    def car_status(self):
        """Summary string for pit wall display."""
        return f"{self.driver_name}: fuel={self.fuel_load:.1f}kg, tire_wear={self.tire_wear:.1f}%"
    
    def complete_lap(self):
        """Drive one more lap."""
        self._laps_driven += 1
    
    @staticmethod
    def kmh_to_mph(kmh):
        """Static method — doesn't need self, just a utility conversion."""
        return kmh * 0.621371
    
    @classmethod
    def get_car_count(cls):
        """Class method — operates on the class, not instance."""
        return cls._car_count
    
    @classmethod
    def from_config(cls, config):
        """Alternative constructor from a config dict."""
        return cls(driver_name=config['driver'], initial_fuel_kg=config.get('fuel', 110))


# Using @property — note NO parentheses!
car = F1Car("Verstappen", initial_fuel_kg=105)
print(f"Initial status: {car.car_status}")

# Drive some laps and watch properties update automatically
for lap in range(10):
    car.complete_lap()
print(f"After 10 laps: {car.car_status}")
print(f"Tire wear: {car.tire_wear:.1f}%")  # Not tire_wear()!
print(f"Fuel remaining: {car.fuel_load:.1f}kg")

# Using @staticmethod — can call on class or instance
print(f"\n350 km/h = {F1Car.kmh_to_mph(350):.1f} mph")

# Using @classmethod
car2 = F1Car("Perez")
print(f"\nTotal cars on grid: {F1Car.get_car_count()}")

# Alternative constructor — build car from a setup sheet
setup_sheet = {'driver': 'Norris', 'fuel': 100}
car3 = F1Car.from_config(setup_sheet)
print(f"Created from config: {car3.driver_name}, fuel={car3.fuel_load:.1f}kg")

---

## 5. Context Managers

Context managers handle setup and cleanup automatically using the `with` statement.

### Why Context Managers Matter in Deep Learning

```python
# Disable gradients during inference
with torch.no_grad():
    predictions = model(test_data)

# Automatic mixed precision training
with torch.cuda.amp.autocast():
    output = model(input)

# Timer context
with Timer("Training"):
    train_one_epoch()
```

### The F1 Connection

**F1 analogy:** A context manager is like a race session. When you enter `with RaceSession("Qualifying"):`, the system automatically: starts the clock, enables telemetry recording, sets the pit lane speed limit, and opens the track. When the session ends (you exit the `with` block), it automatically: waves the checkered flag, stops telemetry, closes the pit lane, and saves all data. You don't have to remember to do any of this cleanup manually — it happens automatically, even if something goes wrong (like a red flag).

In [ ]:
# Context manager for timing — like an F1 session timer
class SessionTimer:
    """Context manager for timing code blocks — like an F1 session clock."""
    
    def __init__(self, session_name="Session"):
        self.session_name = session_name
        
    def __enter__(self):
        """Called when entering the 'with' block — green flag!"""
        self.start = time.time()
        return self  # This is what 'as' binds to
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        """Called when exiting the 'with' block — checkered flag!"""
        self.elapsed = time.time() - self.start
        print(f"{self.session_name} session time: {self.elapsed:.4f} seconds")
        return False  # Don't suppress exceptions


# Use it — like running different F1 sessions
with SessionTimer("Telemetry Processing"):
    A = np.random.randn(500, 500)
    B = np.random.randn(500, 500)
    C = A @ B
    
with SessionTimer("Strategy Simulation"):
    total = 0
    for i in range(100000):
        total += i

In [ ]:
# Simulating torch.no_grad() — like switching to "race mode" (no data logging overhead)
class RaceMode:
    """Context manager to disable telemetry logging (like torch.no_grad())."""
    
    # Class variable to track global state
    logging_enabled = True
    
    def __enter__(self):
        self.prev_state = RaceMode.logging_enabled
        RaceMode.logging_enabled = False
        print("Telemetry logging disabled (race mode — maximum speed)")
        return self
    
    def __exit__(self, *args):
        RaceMode.logging_enabled = self.prev_state
        print("Telemetry logging restored")
        return False


def process_lap_data():
    """Function that checks whether we're logging telemetry."""
    if RaceMode.logging_enabled:
        print("  Processing with full telemetry logging (training mode)")
    else:
        print("  Processing WITHOUT logging — faster! (inference/race mode)")


print("Practice session (training mode):")
process_lap_data()

print("\nQualifying hot lap (race mode — no logging overhead):")
with RaceMode():
    process_lap_data()

print("\nBack to practice (training mode):")
process_lap_data()

---

## 6. Type Hints

Type hints document what types your functions expect and return. They don't enforce types at runtime but help with:
- Documentation
- IDE autocomplete
- Static analysis tools (mypy)

**F1 analogy:** F1 is a safety-critical system. The FIA mandates precise specifications: fuel flow must be a `float` in kg/s, tire pressure must be a `float` in PSI, pit speed limit is an `int` in km/h. Type hints in code serve the same purpose — they're the technical regulations for your data. In a real-time telemetry system, sending a `str` where a `float` is expected could mean the difference between a perfect pit stop and a catastrophic failure. Type hints are how you catch those bugs before they hit the track.

### Basic Type Hints

In [ ]:
from typing import List, Dict, Tuple, Optional, Union, Callable
import numpy as np

# Basic types — strict like FIA technical regulations
def format_driver_name(name: str) -> str:
    return f"Driver: {name}"

def calculate_gap(leader_time: float, chaser_time: float) -> float:
    return chaser_time - leader_time

# Collections — telemetry is full of typed collections
def total_stint_time(lap_times: List[float]) -> float:
    return sum(lap_times)

def get_sensor_shapes(telemetry: Dict[str, np.ndarray]) -> List[Tuple[int, ...]]:
    return [v.shape for v in telemetry.values()]

# Optional (can be None) — like optional DRS activation
def get_tire_response(compound: Optional[str] = None) -> Callable:
    if compound is None or compound == 'soft':
        return lambda x: np.maximum(0, x)  # ReLU (aggressive)
    elif compound == 'medium':
        return lambda x: 1 / (1 + np.exp(-x))  # Sigmoid (balanced)
    else:
        raise ValueError(f"Unknown compound: {compound}")

# Union (can be multiple types) — sensor data arrives in different formats
def normalize_telemetry(data: Union[List[float], np.ndarray]) -> np.ndarray:
    arr = np.array(data)
    return (arr - arr.mean()) / arr.std()


# Examples
print(format_driver_name("Verstappen"))
print(f"Gap: {calculate_gap(91.2, 91.8):.3f}s")
print(f"Total stint: {total_stint_time([91.2, 90.8, 91.5])}s")
print(f"Normalized: {normalize_telemetry([320, 315, 322, 318, 325])}")

In [ ]:
# Type hints in classes — F1 race configuration (like a setup sheet)
from dataclasses import dataclass
from typing import List, Optional


@dataclass
class RaceStrategyConfig:
    """Configuration for a race strategy — like a pre-race setup sheet.
    
    In ML, this pattern is used for TrainingConfig (learning_rate, batch_size, etc.).
    In F1, this is your race engineer's strategy briefing."""
    fuel_load_kg: float = 105.0
    tire_compound: str = "Medium"
    pit_stops: int = 1
    stint_lengths: List[int] = None
    drs_offset: Optional[float] = None
    
    def __post_init__(self):
        if self.stint_lengths is None:
            self.stint_lengths = [25, 32]  # Default: 2-stop strategy


# @dataclass automatically generates __init__, __repr__, etc.
strategy = RaceStrategyConfig(fuel_load_kg=100.0, pit_stops=2)
print(strategy)

# Access fields — just like a training config
print(f"\nFuel load: {strategy.fuel_load_kg}kg")
print(f"Stint lengths: {strategy.stint_lengths} laps")

---

## 7. Putting It All Together: A Mini Deep Learning Framework

Let's build a minimal neural network framework using everything we've learned — with F1 naming throughout.

**F1 analogy:** This is like building a complete car from components. We need a base `Module` (the chassis), `Linear` layers (processing stages in the ECU), activation functions (tire compound response curves), and a `Sequential` container (the full car assembly). Every part uses the OOP concepts we've covered: inheritance, magic methods, properties, and type hints.

In [ ]:
from typing import List, Callable, Optional
from abc import ABC, abstractmethod
import numpy as np


class Module(ABC):
    """Base class for all neural network modules — the chassis of our F1 car."""
    
    def __init__(self):
        self._parameters: Dict[str, np.ndarray] = {}
        self._modules: Dict[str, 'Module'] = {}
        self.training: bool = True  # True = practice session, False = race mode
        
    @abstractmethod
    def forward(self, x: np.ndarray) -> np.ndarray:
        """Forward pass — must be implemented by subclasses (like each team's aero design)."""
        pass
    
    def __call__(self, x: np.ndarray) -> np.ndarray:
        """Make module callable — send the car onto the track."""
        return self.forward(x)
    
    def parameters(self) -> List[np.ndarray]:
        """Return all tunable parameters — like all the adjustable settings in the ECU."""
        params = list(self._parameters.values())
        for module in self._modules.values():
            params.extend(module.parameters())
        return params
    
    def train(self, mode: bool = True):
        """Set training mode — practice session with full telemetry."""
        self.training = mode
        for module in self._modules.values():
            module.train(mode)
        return self
    
    def eval(self):
        """Set evaluation mode — race mode, no logging overhead."""
        return self.train(False)


class Linear(Module):
    """Fully connected layer — a single processing stage in the car's ECU."""
    
    def __init__(self, in_features: int, out_features: int):
        super().__init__()
        # Xavier initialization — like calibrating sensor mappings
        scale = np.sqrt(2.0 / (in_features + out_features))
        self._parameters['weight'] = np.random.randn(in_features, out_features) * scale
        self._parameters['bias'] = np.zeros(out_features)
        
    def forward(self, x: np.ndarray) -> np.ndarray:
        return x @ self._parameters['weight'] + self._parameters['bias']
    
    def __repr__(self):
        w = self._parameters['weight']
        return f"Linear({w.shape[0]}, {w.shape[1]})"


class ReLU(Module):
    """ReLU activation — the Soft tire response: aggressive, all-or-nothing."""
    
    def forward(self, x: np.ndarray) -> np.ndarray:
        return np.maximum(0, x)
    
    def __repr__(self):
        return "ReLU()"


class Sequential(Module):
    """Sequential container — the full car assembly, processing data through each stage."""
    
    def __init__(self, *modules: Module):
        super().__init__()
        for i, module in enumerate(modules):
            self._modules[str(i)] = module
            
    def forward(self, x: np.ndarray) -> np.ndarray:
        for module in self._modules.values():
            x = module(x)
        return x
    
    def __repr__(self):
        """Pit wall display — shows the full car architecture."""
        lines = ["Sequential("]
        for name, module in self._modules.items():
            lines.append(f"  ({name}): {module}")
        lines.append(")")
        return "\n".join(lines)


# Build a telemetry processing model — like assembling an F1 car!
model = Sequential(
    Linear(784, 256),   # Sensor intake: 784 raw channels -> 256 processed
    ReLU(),             # Soft tire response (aggressive activation)
    Linear(256, 128),   # Feature extraction: 256 -> 128
    ReLU(),
    Linear(128, 10)     # Final prediction: 128 -> 10 output classes
)

print(model)
print(f"\nNumber of parameter arrays: {len(model.parameters())}")
print(f"Total parameters: {sum(p.size for p in model.parameters()):,}")

# Forward pass — process a batch of telemetry snapshots
x = np.random.randn(32, 784)  # 32 telemetry snapshots, 784 sensors each
output = model(x)
print(f"\nInput shape (snapshots x sensors): {x.shape}")
print(f"Output shape (snapshots x predictions): {output.shape}")

---

## Exercises

### Exercise 1: Build a DRS (Drag Reduction System) Layer

In F1, DRS opens a flap on the rear wing to reduce drag on straights — but only when you're within 1 second of the car ahead, and only in designated DRS zones.

In neural networks, **Dropout** works similarly: it randomly "opens" (zeros out) neurons during training to prevent over-reliance on any single pathway. During the race (evaluation), DRS rules are off — all neurons are active.

Implement a dropout layer that:
- During training: randomly zeros elements with probability `p` (like randomly disabling DRS for some cars)
- During evaluation: does nothing (but scales output to compensate)

In [ ]:
class DRSLayer(Module):
    """Dropout layer — like the DRS system randomly enabling/disabling aero elements."""
    
    def __init__(self, p: float = 0.5):
        super().__init__()
        self.p = p  # Probability of dropping a neuron (closing DRS)
        
    def forward(self, x: np.ndarray) -> np.ndarray:
        # TODO: Implement dropout
        # Hint: 
        # - During training (self.training == True): create mask, apply it, scale by 1/(1-p)
        # - During eval (race day): just return x — all systems active
        
        if self.training:
            # Like randomly activating DRS for (1-p) fraction of the field
            mask = np.random.binomial(1, 1 - self.p, x.shape)
            return x * mask / (1 - self.p)  # Scale to keep expected value the same
        return x
    
    def __repr__(self):
        return f"DRSLayer(p={self.p})"


# Test the DRS layer
drs = DRSLayer(p=0.5)
x = np.ones((2, 10))  # 2 cars, 10 sensor readings each

print("Practice session (training mode — DRS randomly applied):")
drs.train()
print(drs(x))

print("\nRace day (eval mode — all systems active):")
drs.eval()
print(drs(x))

### Interactive Example: How DRS / Dropout Rate Affects Performance

Try changing the dropout rate to see how it affects the network's activations — like adjusting how aggressively the DRS zones are enforced.

In [ ]:
# Interactive: Visualizing dropout at different rates — like DRS activation zones
# Try changing dropout_rate to see the effect!

def visualize_drs_effect(dropout_rates=[0.0, 0.25, 0.5, 0.75]):
    """Visualize how different dropout rates affect sensor activations."""
    np.random.seed(42)
    
    # Create input (simulating sensor readings from a hidden layer)
    x = np.abs(np.random.randn(1, 100))  # 100 sensor channels, positive values
    
    fig, axes = plt.subplots(2, 2, figsize=(12, 8))
    axes = axes.flatten()
    
    for ax, p in zip(axes, dropout_rates):
        # Apply dropout (DRS-style random masking)
        if p > 0:
            mask = np.random.binomial(1, 1 - p, x.shape)
            output = x * mask / (1 - p)  # Inverted dropout scaling
        else:
            output = x.copy()
            mask = np.ones_like(x)
        
        # Visualize — green = active sensor, red = dropped
        colors = ['green' if m else 'red' for m in mask.flatten()]
        ax.bar(range(100), output.flatten(), color=colors, alpha=0.7, width=1.0)
        ax.axhline(y=x.mean(), color='blue', linestyle='--', 
                   label=f'Original mean: {x.mean():.2f}', linewidth=2)
        ax.axhline(y=output.mean(), color='orange', linestyle='-', 
                   label=f'After DRS/dropout: {output.mean():.2f}', linewidth=2)
        
        n_dropped = int(p * 100)
        ax.set_title(f'DRS Rate p={p}\n({n_dropped} sensors dropped, {100-n_dropped} active)', 
                     fontsize=11, fontweight='bold')
        ax.set_xlabel('Sensor channel')
        ax.set_ylabel('Activation value')
        ax.legend(loc='upper right', fontsize=8)
        ax.set_xlim(-1, 101)
    
    plt.suptitle('Effect of Dropout/DRS Rate on Sensor Activations\n(Green = active, Red = dropped)', 
                 fontsize=13, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.show()
    
    print("Key insight: Notice how the mean stays roughly the same!")
    print("This is because we scale by 1/(1-p) during training.")
    print("In F1 terms: the team compensates for DRS by adjusting other aero elements.")
    print("\nTry different rates:")
    print("  - p=0.0: No dropout (all sensors active — like a dry race)")
    print("  - p=0.5: Typical dropout (50% dropped — balanced regulation)")
    print("  - p=0.75: Aggressive dropout (may hurt — like heavy rain)")

# Run the visualization
visualize_drs_effect()

### Exercise 2: Create a Race Data Loader

In F1, telemetry data streams in continuously. But the pit wall processes it in batches — chunks of laps at a time, not one sample at a time.

Build a DataLoader (like a pit wall data feed) that:
- Takes a dataset and batch_size
- Is iterable (use `__iter__` and `__next__` — iterate through batches of laps)
- Optionally shuffles data (like randomizing which laps to analyze first)

In [ ]:
class PitWallDataLoader:
    """DataLoader — like the pit wall data feed that processes telemetry in batches."""
    
    def __init__(self, dataset, batch_size: int = 32, shuffle: bool = False):
        self.dataset = dataset
        self.batch_size = batch_size
        self.shuffle = shuffle
        
    def __len__(self):
        """Number of batches — how many chunks of data to process."""
        return (len(self.dataset) + self.batch_size - 1) // self.batch_size
    
    def __iter__(self):
        """Return iterator — start a new pass through the data."""
        # TODO: Create indices, optionally shuffle, reset position
        self.indices = np.arange(len(self.dataset))
        if self.shuffle:
            np.random.shuffle(self.indices)  # Randomize lap order for training
        self.pos = 0
        return self
    
    def __next__(self):
        """Get next batch of laps — like the pit wall requesting the next chunk."""
        # TODO: Return next batch or raise StopIteration (checkered flag!)
        if self.pos >= len(self.dataset):
            raise StopIteration  # Checkered flag — no more data
            
        batch_indices = self.indices[self.pos:self.pos + self.batch_size]
        self.pos += self.batch_size
        
        # Collect batch of telemetry and labels
        batch_telemetry = []
        batch_compounds = []
        for idx in batch_indices:
            telemetry, compound = self.dataset[idx]
            batch_telemetry.append(telemetry)
            batch_compounds.append(compound)
            
        return np.array(batch_telemetry), np.array(batch_compounds)


# Test with our season telemetry dataset
season_data = SeasonTelemetryDataset(100, 10, 3)
pit_wall_feed = PitWallDataLoader(season_data, batch_size=32, shuffle=True)

print(f"Total laps in dataset: {len(season_data)}")
print(f"Number of batches: {len(pit_wall_feed)}")

print("\nPit wall processing batches:")
for i, (telemetry, compounds) in enumerate(pit_wall_feed):
    print(f"  Batch {i}: telemetry.shape={telemetry.shape}, compounds.shape={compounds.shape}")

---

## Summary

### Key Concepts

| Concept | What It Does | PyTorch Example | F1 Parallel |
|---------|--------------|------------------|-------------|
| Classes | Bundle data and methods | `nn.Module`, `Dataset` | Car design spec (aero, engine, suspension) |
| Inheritance | Extend existing classes | `class MyModel(nn.Module)` | SoftTire/MediumTire/HardTire extend base Tire |
| `__init__` | Initialize object | Setup layers and parameters | Factory build: install engine, tires, aero |
| `__call__` | Make callable | `model(input)` | Send the car onto the track |
| `__len__` | Enable `len()` | `len(dataset)` | Number of laps in a stint |
| `__getitem__` | Enable indexing | `dataset[0]` | Pull up lap 17's telemetry |
| `__repr__` | Nice printing | `print(model)` | Pit wall display |
| `__lt__` | Comparison | `result1 < result2` | Championship standings |
| `__add__` | Addition | `tensor1 + tensor2` | Combining stint times |
| `@property` | Computed attributes | `model.device` | `car.tire_wear` (computed from laps) |
| `@timed` | Custom decorator | Profiling | Sector timing loops |
| Decorators | Modify functions | `@torch.no_grad()` | FIA scrutineering (validate without changing) |
| Context managers | Setup/cleanup | `with torch.no_grad():` | Race session (green flag to checkered flag) |
| Type hints | Document types | `def forward(self, x: Tensor) -> Tensor:` | FIA technical regulations (strict specs) |

### Checklist
- [ ] I can create classes with `__init__` and methods
- [ ] I understand inheritance and `super()` (base Tire -> SoftTire, MediumTire, HardTire)
- [ ] I can use magic methods (`__call__`, `__len__`, `__getitem__`, `__lt__`, `__add__`)
- [ ] I can write and use decorators (`@timed`, `@validated`)
- [ ] I can create context managers (session timers, race mode)
- [ ] I can add type hints to functions and classes (safety-critical specs)

### Connection to Deep Learning

| Concept | PyTorch Example | Why It Matters | F1 Parallel |
|---------|-----------------|----------------|-------------|
| **Classes & `__init__`** | `class MyModel(nn.Module): def __init__(self): ...` | Every neural network is a class; `__init__` sets up layers and registers parameters | Every F1 car is an instance; the factory build installs engine, aero, and suspension |
| **Inheritance** | `class MyModel(nn.Module)` | You inherit GPU support, parameter tracking, save/load, and gradient computation for free | A SoftTire inherits base rubber chemistry and only overrides grip characteristics |
| **`__call__` & `forward`** | `output = model(x)` calls `forward()` | PyTorch adds hooks and gradient tracking when you call the model, so always use `model(x)` not `model.forward(x)` | Sending the car onto track triggers pre-checks, the lap, and post-lap data — not just driving |
| **`__len__` & `__getitem__`** | `class MyDataset(Dataset)` | DataLoader uses these to batch and shuffle your data automatically | The pit wall uses these to pull any lap's telemetry by index and process in batches |
| **`@property`** | `model.device`, `tensor.shape` | Access computed values cleanly without parentheses | `car.tire_wear` and `car.fuel_load` — always current, always computed from state |
| **`@torch.no_grad()`** | `with torch.no_grad(): pred = model(x)` | Disables gradient tracking during inference for speed and memory savings | Race mode: disable telemetry logging overhead for maximum performance |
| **Context managers** | `with autocast(): ...` | Mixed precision training, gradient scaling, and resource management | Race sessions: automatic setup (green flag) and cleanup (checkered flag) |
| **Type hints** | `def forward(self, x: Tensor) -> Tensor` | IDE autocomplete, better documentation, catch bugs early | FIA technical regulations: strict type specs prevent safety-critical errors |

---

## Next Steps

Continue to **Part 2.2: NumPy Deep Dive** where we'll cover:
- Advanced array operations (processing multi-channel telemetry data)
- Broadcasting in depth (applying fuel corrections across all laps simultaneously)
- Vectorization for performance (why the pit wall processes matrices, not loops)